In [1]:
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize


## Creating the news dataframe
The first step is import the news dataset and creating a Pandas Dataframe

In [2]:
news_data_file = open('../datasets/raw-data/folha_de_sao_paulo_news.json', 'r')
news_json = json.load(news_data_file)

news_df = json_normalize(news_json['news'])

news_df.head(5)

,authors,categories,date_published,link,locations,news_body,sub_title,title
0,[talita fernandes],"[poder, governo_bolsonaro, lava_jato, legislat...",2019-05-28 22:40:00,https://www1.folha.uol.com.br/poder/2019/05/bo...,[brasília],", jair bolsonaro (psl) afirmou ter mais poder ...","durante evento, presidente comentou encontro q...",bolsonaro diz ter mais poder que maia por ter ...
1,"[julio wiziack, gustavo uribe]","[mercado, mercado_financeiro, previdência, tec...",2019-05-28 12:46:00,https://www1.folha.uol.com.br/mercado/2019/05/...,[brasília],confederações patronais que não estão alinhada...,estamos trabalhando para que ela seja aprovada...,"por corte menor, sistema s promete a bolsonaro..."
2,[daniel carvalho],"[poder, governo_bolsonaro, lava_jato, legislat...",2019-05-28 11:16:00,https://www1.folha.uol.com.br/poder/2019/05/ma...,[brasília],"o líder do psl no senado, major olímpio (sp), ...",ministro da justiça telefonou ao senador para ...,"major olímpio sobre moro: ""não dá para defende..."
3,[carolina linhares],"[poder, governo_bolsonaro, lava_jato, legislat...",2019-05-28 02:00:00,https://www1.folha.uol.com.br/poder/2019/05/do...,[são_paulo],"na última sexta (24), quando o afirmou que est...","alvo de processo de expulsão, fernando guimarã...",doria é stalinista ao dizer que quem não conco...
4,[NO AUTHOR],"[poder, governo_bolsonaro, lava_jato, legislat...",2019-05-28 10:24:00,https://www1.folha.uol.com.br/poder/2019/05/pl...,[brasília],o presidente jair bolsonaro discutiu nesta ter...,"segundo esboço de documento, texto destaca ref...","planalto, congresso e stf combinam assinar pac..."


# Selecting only the columns that we are going to feed into the Neural Network

In [3]:
news_df['text'] = news_df['title'] + '. ' + news_df['sub_title'] + '. ' + news_df['news_body']
#news_df['text'] = news_df['title']
news_df = news_df[['date_published', 'text']]

news_df.head(5)

,date_published,text
0,2019-05-28 22:40:00,bolsonaro diz ter mais poder que maia por ter ...
1,2019-05-28 12:46:00,"por corte menor, sistema s promete a bolsonaro..."
2,2019-05-28 11:16:00,"major olímpio sobre moro: ""não dá para defende..."
3,2019-05-28 02:00:00,doria é stalinista ao dizer que quem não conco...
4,2019-05-28 10:24:00,"planalto, congresso e stf combinam assinar pac..."


# Saving the clean dataset into a file

In [4]:
news_df.to_csv('../datasets/pre-processed/news.csv', index= False, encoding='utf-8')